 # notebook description
In this notebook, I appropriately adapted the data structures from the source files to a format compatible with `Postgres`, and then uploaded them to the server.

In [1]:
import psycopg2
from psycopg2 import connect
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import pandas as pd

In [2]:
username = 'DB_ID'
password = 'DB_PASS'
host = 'localhost'
database = 'airlines'
default_database = 'postgres'
port = 5432

In [4]:
url = URL.create(
    "postgresql+psycopg2",
    database = database,
    username=username,
    password=password,
    host=host,
    port=port    
    )
engine = create_engine(url)

In [5]:
def load_raw_data(file_name):
    df = pd.read_csv(file_name)
    
    df.columns = [col.lower() for col in df.columns]
    
    return df

In [6]:
# Loading individual files into frames
# File paths
api_data_path = "./data/apidata"
aircraft_file_path = f'{api_data_path}/aircraft.csv'
airport_list_file_path = f'{api_data_path}/airport_list.csv'
airport_weather_file_path = f'{api_data_path}/airport_weather.csv'
flight_file_path = f'{api_data_path}/flight.csv'

# Loading files
aircraft_df = load_raw_data(aircraft_file_path)
airport_list_df = load_raw_data(airport_list_file_path)
airport_weather_df = load_raw_data(airport_weather_file_path)
flight_df = load_raw_data(flight_file_path)

aircraft_df.head()

,manufacture_year,tail_num,number_of_seats
0,1944,N54514,0.0
1,1945,N1651M,0.0
2,1953,N100CE,0.0
3,1953,N141FL,0.0
4,1953,N151FL,0.0


In [8]:
# Data export function
def export_table_to_db(df, table_name):
    print(f"Loading data into {table_name}...")

    df.reset_index(drop=True, inplace=True)
    df.index = df.index + 1

    try:
        global engine
        
        df.to_sql(name=table_name, con=engine, if_exists='replace', index=True, index_label='id', chunksize=500)
        print(f"Data successfully loaded into {table_name}.")
    except Exception as e:
        print(f"Failed to load data into {table_name}: {e}")

In [9]:
# Export first 5 rows for test
export_table_to_db(aircraft_df.head(5), 'aircraft')

# If the test succeeds, load the full data
export_table_to_db(aircraft_df, 'aircraft')

Loading data into aircraft...
Data successfully loaded into aircraft.
Loading data into aircraft...
Data successfully loaded into aircraft.


In [10]:
export_table_to_db(airport_weather_df.head(5), 'airport_weather')

export_table_to_db(airport_weather_df, 'airport_weather')

Loading data into airport_weather...
Data successfully loaded into airport_weather.
Loading data into airport_weather...
Data successfully loaded into airport_weather.


In [11]:
export_table_to_db(flight_df.head(5), 'flight')

export_table_to_db(flight_df, 'flight')

Loading data into flight...
Data successfully loaded into flight.
Loading data into flight...
Data successfully loaded into flight.


In [12]:
export_table_to_db(airport_list_df.head(5), 'airport_list')

export_table_to_db(airport_list_df, 'airport_list')

Loading data into airport_list...
Data successfully loaded into airport_list.
Loading data into airport_list...
Data successfully loaded into airport_list.
